In [124]:
# https://www.youtube.com/watch?v=U0s0f995w14
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange, tqdm
import math

# Transformer

In [146]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embed size must be div by heads"

        self.value = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.key = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.query = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, value, key, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = value.shape[1], key.shape[1], query.shape[1]

        # Split embed to self.heads piceses
        value = value.reshape(N, value_len, self.heads, self.head_dim)
        key = key.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        value = self.value(value)
        key = self.key(key)
        query = self.query(query)

        energy = torch.einsum("nqhd, nkhd-> nhqk", [query, key])
        # energy shape: (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql, nlhd->nqhd", [attention, value]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        out = self.fc_out(out)
        return out
        # (N, query_len, heads, head_dim)

In [147]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feedforward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feedforward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [148]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        n_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_len,
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size

        # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_len, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(n_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_len = x.shape
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [149]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.dropout = nn.Dropout(dropout)
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )

    def forward(self, x, value, key, src_mask, tar_mask):
        attention = self.attention(x, x, x, tar_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [165]:
class Decoder(nn.Module):
    def __init__(
        self,
        tar_vocab_size,
        embed_size,
        n_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_len,
    ):
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(tar_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_len, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(n_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, tar_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encode_out, src_mask, tar_mask):
        N, seq_len = x.shape
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)

        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, encode_out, encode_out, src_mask, tar_mask)
        out = self.fc_out(out)
        return out

In [166]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tar_vocab_size,
        src_pad_idx,
        tar_pad_idx,
        embed_size=256,
        n_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cuda",
        max_len=100,
    ):

        super(Transformer, self).__init__()
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            n_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_len,
        )

        self.decoder = Decoder(
            tar_vocab_size,
            embed_size,
            n_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_len,
        )

        self.src_pad_idx = src_pad_idx
        self.tar_pad_idx = tar_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_tar_mask(self, tar):
        N, tar_len = tar.shape
        tar_mask = torch.tril(torch.ones((tar_len, tar_len))).expand(
            N, 1, tar_len, tar_len
        )
        return tar_mask.to(self.device)

    def forward(self, src, tar):
        src_mask = self.make_src_mask(src)
        tar_mask = self.make_tar_mask(tar)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(tar, enc_src, src_mask, tar_mask)
        return out

# Example

In [167]:
device = torch.device("cuda")

x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)

tar = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0,], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

src_pad_idx = 0
tar_pad_idx = 0
src_vocab_size = 10
tar_vocab_size = 10

model = Transformer(src_vocab_size, tar_vocab_size, src_pad_idx, tar_pad_idx).to(device)

In [170]:
tar[:, :-1]

tensor([[1, 7, 4, 3, 5, 9, 2],
        [1, 5, 6, 2, 4, 7, 6]], device='cuda:0')

In [171]:
out = model(x, tar[:, :-1])
out.shape

torch.Size([2, 7, 10])

In [174]:
a1 = 4